In [11]:
import pandas as pd
import numpy as np

from sklearn.neural_network import MLPClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder


def data_cleaning(df):
    df['Sex'] = list(map(lambda x: 'M' if x==1 else 'F', df['Sex']))
    #Transforming into category, thus 0 and 1 doesn't have distance interpretation on sex
    
    df['Chest pain type'] = list(map(lambda x: 'Light' if x==1 else 'Common' if x==2 else 'Hard' if x==3 else 'Dangerous', df['Chest pain type']))
    #Transforming into category, same reason
    
    df['FBS over 120'] = list(map(lambda x: 'Y' if x==1 else 'N', df['FBS over 120']))
    #Transforming into category, same reason
    
    df['Exercise angina'] = list(map(lambda x: 'Y' if x==1 else 'N', df['Exercise angina']))
    # Transforming into category, same reason
    
    df['Slope of ST'] = list(map(lambda x: 'Upsloping' if x==1 else 'Flat' if x==2 else 'Downsloping', df['Slope of ST']))
    # Transforming into category, same reason
    
    df['EKG results'] = list(map(lambda x: 'N' if x in [0,1] else 'P', df['EKG results']))
    # Merging 0 and 1 to 'N' thus 0 and 1 have the same relation with Heart Disease and, transforming into category
    # thus, here also, there's no distance relation between 0, 1 and 2
    return df


In [7]:
target = 'Heart Disease'
df = pd.read_csv('df_att.csv')
X = df.drop(target, axis=1)
y = df[target]

cat_features = X.select_dtypes(exclude='number').columns.tolist()
num_features = X.select_dtypes(include='number').columns.tolist()

test = pd.read_csv('test.csv')

test = data_cleaning(test)

test

,id,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium
0,630000,58,M,Hard,120,288,N,P,145,Y,0.8,Flat,3,3
1,630001,55,F,Common,120,209,N,N,172,N,0.0,Upsloping,0,3
2,630002,54,M,Dangerous,120,268,N,N,150,Y,0.0,Flat,3,7
3,630003,44,F,Hard,112,177,N,N,168,N,0.9,Upsloping,0,3
4,630004,43,M,Light,138,267,N,N,163,N,1.8,Flat,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269995,899995,58,M,Common,120,222,N,N,172,N,1.0,Upsloping,0,7
269996,899996,58,M,Dangerous,132,289,N,N,172,N,2.8,Flat,0,3
269997,899997,63,M,Hard,108,201,Y,N,158,N,0.8,Upsloping,0,3
269998,899998,59,M,Dangerous,120,274,N,P,163,N,0.5,Upsloping,0,3


In [12]:
preprocessor = ColumnTransformer(
    transformers =[
        ("num", StandardScaler(), num_features),
        ("cat", OneHotEncoder(handle_unknown='ignore'), cat_features)
    ]
)

preprocessor.fit(X)
X = preprocessor.transform(X)

test = preprocessor.transform(test)
X, test

(array([[ 0.46792102,  1.43582185, -0.17848981, ...,  0.        ,
          1.        ,  0.        ],
        [-0.25879726, -0.36708803,  2.37483653, ...,  0.        ,
          0.        ,  1.        ],
        [ 0.22568159,  1.97001737, -1.69267171, ...,  0.        ,
          0.        ,  1.        ],
        ...,
        [ 1.55799844, -0.03321583,  0.89034447, ...,  0.        ,
          0.        ,  1.        ],
        [-0.25879726,  0.63452857, -1.36608346, ...,  0.        ,
          0.        ,  1.        ],
        [-0.37991697, -0.03321583, -1.36608346, ...,  0.        ,
          0.        ,  1.        ]], shape=(630000, 22)),
 array([[ 0.46792102, -0.70096023,  1.27631241, ...,  0.        ,
          1.        ,  0.        ],
        [ 0.10456188, -0.70096023, -1.06918505, ...,  0.        ,
          0.        ,  1.        ],
        [-0.01655783, -0.70096023,  0.68251558, ...,  0.        ,
          1.        ,  0.        ],
        ...,
        [ 1.07351959, -1.50225352,

In [15]:
params = {'hidden_layer_sizes': (128,), 'alpha': 5.894589851301599e-05, 'learning_rate_init': 0.0007594934857959463, 'activation': 'relu'}

In [16]:
model = MLPClassifier(**params)
model

,"hidden_layer_sizes hidden_layer_sizes: array-like of shape(n_layers - 2,), default=(100,)The ith element represents the number of neurons in the ithhidden layer.","(128,)"
,"activation activation: {'identity', 'logistic', 'tanh', 'relu'}, default='relu'Activation function for the hidden layer.- 'identity', no-op activation, useful to implement linear bottleneck, returns f(x) = x- 'logistic', the logistic sigmoid function, returns f(x) = 1 / (1 + exp(-x)).- 'tanh', the hyperbolic tan function, returns f(x) = tanh(x).- 'relu', the rectified linear unit function, returns f(x) = max(0, x)",'relu'
,"solver solver: {'lbfgs', 'sgd', 'adam'}, default='adam'The solver for weight optimization.- 'lbfgs' is an optimizer in the family of quasi-Newton methods.- 'sgd' refers to stochastic gradient descent.- 'adam' refers to a stochastic gradient-based optimizer proposed by Kingma, Diederik, and Jimmy BaFor a comparison between Adam optimizer and SGD, see:ref:`sphx_glr_auto_examples_neural_networks_plot_mlp_training_curves.py`.Note: The default solver 'adam' works pretty well on relativelylarge datasets (with thousands of training samples or more) in terms ofboth training time and validation score.For small datasets, however, 'lbfgs' can converge faster and performbetter.",'adam'
,"alpha alpha: float, default=0.0001Strength of the L2 regularization term. The L2 regularization termis divided by the sample size when added to the loss.For an example usage and visualization of varying regularization, see:ref:`sphx_glr_auto_examples_neural_networks_plot_mlp_alpha.py`.",5.894589851301599e-05
,"batch_size batch_size: int, default='auto'Size of minibatches for stochastic optimizers.If the solver is 'lbfgs', the classifier will not use minibatch.When set to ""auto"", `batch_size=min(200, n_samples)`.",'auto'
,"learning_rate learning_rate: {'constant', 'invscaling', 'adaptive'}, default='constant'Learning rate schedule for weight updates.- 'constant' is a constant learning rate given by 'learning_rate_init'.- 'invscaling' gradually decreases the learning rate at each time step 't' using an inverse scaling exponent of 'power_t'. effective_learning_rate = learning_rate_init / pow(t, power_t)- 'adaptive' keeps the learning rate constant to 'learning_rate_init' as long as training loss keeps decreasing. Each time two consecutive epochs fail to decrease training loss by at least tol, or fail to increase validation score by at least tol if 'early_stopping' is on, the current learning rate is divided by 5.Only used when ``solver='sgd'``.",'constant'
,"learning_rate_init learning_rate_init: float, default=0.001The initial learning rate used. It controls the step-sizein updating the weights. Only used when solver='sgd' or 'adam'.",0.0007594934857959463
,"power_t power_t: float, default=0.5The exponent for inverse scaling learning rate.It is used in updating effective learning rate when the learning_rateis set to 'invscaling'. Only used when solver='sgd'.",0.5
,"max_iter max_iter: int, default=200Maximum number of iterations. The solver iterates until convergence(determined by 'tol') or this number of iterations. For stochasticsolvers ('sgd', 'adam'), note that this determines the number of epochs(how many times each data point will be used), not the number ofgradient steps.",200
,"shuffle shuffle: bool, default=TrueWhether to shuffle samples in each iteration. Only used whensolver='sgd' or 'adam'.",True
,"random_state random_state: int, RandomState instance, default=NoneDetermines random number generation for weights and biasinitialization, train-test split if early stopping is used, and batchsampling when solver='sgd' or 'adam'.Pass an int for reproducible results across multiple function calls.See :term:`Glossary `.",None


In [17]:
model.fit(X, y)

,"hidden_layer_sizes hidden_layer_sizes: array-like of shape(n_layers - 2,), default=(100,)The ith element represents the number of neurons in the ithhidden layer.","(128,)"
,"activation activation: {'identity', 'logistic', 'tanh', 'relu'}, default='relu'Activation function for the hidden layer.- 'identity', no-op activation, useful to implement linear bottleneck, returns f(x) = x- 'logistic', the logistic sigmoid function, returns f(x) = 1 / (1 + exp(-x)).- 'tanh', the hyperbolic tan function, returns f(x) = tanh(x).- 'relu', the rectified linear unit function, returns f(x) = max(0, x)",'relu'
,"solver solver: {'lbfgs', 'sgd', 'adam'}, default='adam'The solver for weight optimization.- 'lbfgs' is an optimizer in the family of quasi-Newton methods.- 'sgd' refers to stochastic gradient descent.- 'adam' refers to a stochastic gradient-based optimizer proposed by Kingma, Diederik, and Jimmy BaFor a comparison between Adam optimizer and SGD, see:ref:`sphx_glr_auto_examples_neural_networks_plot_mlp_training_curves.py`.Note: The default solver 'adam' works pretty well on relativelylarge datasets (with thousands of training samples or more) in terms ofboth training time and validation score.For small datasets, however, 'lbfgs' can converge faster and performbetter.",'adam'
,"alpha alpha: float, default=0.0001Strength of the L2 regularization term. The L2 regularization termis divided by the sample size when added to the loss.For an example usage and visualization of varying regularization, see:ref:`sphx_glr_auto_examples_neural_networks_plot_mlp_alpha.py`.",5.894589851301599e-05
,"batch_size batch_size: int, default='auto'Size of minibatches for stochastic optimizers.If the solver is 'lbfgs', the classifier will not use minibatch.When set to ""auto"", `batch_size=min(200, n_samples)`.",'auto'
,"learning_rate learning_rate: {'constant', 'invscaling', 'adaptive'}, default='constant'Learning rate schedule for weight updates.- 'constant' is a constant learning rate given by 'learning_rate_init'.- 'invscaling' gradually decreases the learning rate at each time step 't' using an inverse scaling exponent of 'power_t'. effective_learning_rate = learning_rate_init / pow(t, power_t)- 'adaptive' keeps the learning rate constant to 'learning_rate_init' as long as training loss keeps decreasing. Each time two consecutive epochs fail to decrease training loss by at least tol, or fail to increase validation score by at least tol if 'early_stopping' is on, the current learning rate is divided by 5.Only used when ``solver='sgd'``.",'constant'
,"learning_rate_init learning_rate_init: float, default=0.001The initial learning rate used. It controls the step-sizein updating the weights. Only used when solver='sgd' or 'adam'.",0.0007594934857959463
,"power_t power_t: float, default=0.5The exponent for inverse scaling learning rate.It is used in updating effective learning rate when the learning_rateis set to 'invscaling'. Only used when solver='sgd'.",0.5
,"max_iter max_iter: int, default=200Maximum number of iterations. The solver iterates until convergence(determined by 'tol') or this number of iterations. For stochasticsolvers ('sgd', 'adam'), note that this determines the number of epochs(how many times each data point will be used), not the number ofgradient steps.",200
,"shuffle shuffle: bool, default=TrueWhether to shuffle samples in each iteration. Only used whensolver='sgd' or 'adam'.",True
,"random_state random_state: int, RandomState instance, default=NoneDetermines random number generation for weights and biasinitialization, train-test split if early stopping is used, and batchsampling when solver='sgd' or 'adam'.Pass an int for reproducible results across multiple function calls.See :term:`Glossary `.",None


In [18]:
MLPCpreds = model.predict_proba(test)[:, 1]
MLPCpreds

array([0.93394057, 0.00404241, 0.99005345, ..., 0.05194783, 0.28453174,
       0.02419166], shape=(270000,))